In [ ]:
!pip install faiss_cpu

In [ ]:
import json
import os
import re
import numpy as np
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
import faiss
import time
import os
import zipfile
import shutil
import glob
import google.generativeai as genai
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import plotly.express as px
import random

In [ ]:
zip_file_path = '/content/dataset.zip'

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall()
    print(f"Zip file '{zip_file_path}' has been extracted successfully.")
except FileNotFoundError:
    print(f"Error: Zip file not found at '{zip_file_path}'")
except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file")
except Exception as e:
    print(f"An error occurred during extraction: {e}")

Zip file '/content/dataset.zip' has been extracted successfully.


In [ ]:
# --- Konfigurasi ---
BASE_DIR = "dataset/SistemOperasi"  # Direktori utama mata kuliah Anda
OUTLINE_FILE = os.path.join(BASE_DIR, "outline_operating_systems.txt")

# --- Path untuk Artefak Model Dasar (Baseline) ---
BASE_EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'
OUTPUT_JSON_CHUNKS_BASE = os.path.join(BASE_DIR, "processed_chunks_metadata_base.json")
OUTPUT_FAISS_INDEX_BASE = os.path.join(BASE_DIR, "vector_store_base.index")

# --- Path untuk Artefak Model Fine-Tuned ---
# Ini akan menjadi output utama yang digunakan aplikasi jika fine-tuning berhasil
FINETUNED_EMBEDDING_MODEL_SAVE_PATH = os.path.join(BASE_DIR, "finetuned_embedding_model_sistem_operasi")
FINETUNED_MODEL_ZIP_PATH = os.path.join(BASE_DIR, "finetuned_embedding_model_sistem_operasi.zip")
OUTPUT_JSON_CHUNKS_FINETUNED = os.path.join(BASE_DIR, "processed_chunks_metadata_finetuned.json")
OUTPUT_FAISS_INDEX_FINETUNED = os.path.join(BASE_DIR, "vector_store_finetuned.index")


# --- File Dataset ---
EMBEDDING_FINETUNING_DATASET_FILE = os.path.join(BASE_DIR, "embedding_finetuning_examples.jsonl")
EMBEDDING_EVALUATION_SET_FILE = os.path.join(BASE_DIR, "embedding_evaluation_set.jsonl")


# --- Parameter ---
MAX_CHUNK_SIZE_CHARS = 1000
CHUNK_OVERLAP_CHARS = 150
HEADING_SPLIT_PATTERN = r"(^#{1,6}\s+.*$)"
FINETUNE_EPOCHS = 3 # Sesuaikan berdasarkan dataset Anda
FINETUNE_BATCH_SIZE = 16
EVALUATION_TOP_K = 5 # Untuk Recall@K dan MRR@K
FINETUNE_BATCH_SIZE = 16

In [ ]:
# --- Fungsi Helper (Parsing, Chunking - sebagian besar sama) ---
def parse_outline(outline_filepath):
    pertemuan_list = []
    try:
        with open(outline_filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        if not content.strip().startswith("MATAKULIAH:"):
            print(f"Peringatan: Format file outline '{outline_filepath}' mungkin tidak sesuai.")
        pertemuan_blocks = re.split(r'\nPERTEMUAN:', '\n' + content.split('PERTEMUAN:', 1)[-1] if 'PERTEMUAN:' in content else '')
        for block in pertemuan_blocks:
            if not block.strip(): continue
            current_pertemuan = {}
            lines = block.strip().splitlines()
            if lines:
                pertemuan_id_match = re.match(r'^\s*(\d+)', lines[0])
                if pertemuan_id_match: current_pertemuan['id'] = int(pertemuan_id_match.group(1))
                else: continue # Skip jika tidak ada ID
                for line_idx, line_content in enumerate(lines): # Ganti nama variabel 'line'
                    if ":" in line_content:
                        key, value = line_content.split(":", 1)
                        key_clean = key.strip().lower().replace(" ", "_")
                        value_clean = value.strip()
                        current_pertemuan[key_clean] = value_clean
            if 'id' in current_pertemuan and 'judul' in current_pertemuan and current_pertemuan.get('file_materi'):
                pertemuan_list.append(current_pertemuan)
            elif 'id' in current_pertemuan:
                 print(f"Info: Pertemuan {current_pertemuan.get('id','N/A')} ({current_pertemuan.get('judul','Tanpa Judul')}) dilewati (mungkin kurang file_materi atau judul).")
    except FileNotFoundError: print(f"Error: File outline '{outline_filepath}' tidak ditemukan.")
    except Exception as e: print(f"Error saat mem-parsing file outline: {e}")
    print(f"Berhasil mem-parsing {len(pertemuan_list)} pertemuan dengan file materi dari outline.")
    return pertemuan_list

def read_material_text(material_filepath):
    try:
        with open(material_filepath, 'r', encoding='utf-8') as f: return f.read()
    except FileNotFoundError: print(f"Error: File materi '{material_filepath}' tidak ditemukan."); return ""
    except Exception as e: print(f"Error saat membaca file materi '{material_filepath}': {e}"); return ""

def _split_text_block_sliding_window(text_block, pertemuan_id, pertemuan_judul, heading, max_size, overlap):
    final_chunks = []
    text_block_stripped = text_block.strip()
    if not text_block_stripped: return []
    if len(text_block_stripped) <= max_size:
        final_chunks.append({"pertemuan_id": pertemuan_id, "pertemuan_judul": pertemuan_judul, "original_heading": heading, "chunk_text": text_block_stripped, "chunk_id": f"p{pertemuan_id}_h{hash(heading)}_{len(final_chunks)}"})
    else:
        start_index = 0; doc_len = len(text_block_stripped)
        while start_index < doc_len:
            end_index = start_index + max_size
            current_slice = text_block_stripped[start_index:min(end_index, doc_len)]
            if current_slice.strip():
                final_chunks.append({"pertemuan_id": pertemuan_id, "pertemuan_judul": pertemuan_judul, "original_heading": heading, "chunk_text": current_slice.strip(), "chunk_id": f"p{pertemuan_id}_h{hash(heading)}_{len(final_chunks)}"})
            if min(end_index, doc_len) >= doc_len: break
            start_index += (max_size - overlap)
            if start_index >= doc_len: break
    return final_chunks

def chunk_material_heading_aware(text_content, pertemuan_id, pertemuan_judul):
    processed_chunks = []; current_chunk_idx = 0
    if not text_content or not text_content.strip(): return []
    parts = re.split(HEADING_SPLIT_PATTERN, text_content, flags=re.MULTILINE)
    current_heading = "Umum"; accumulated_text_for_section = ""
    for i, part in enumerate(parts):
        part_stripped = part.strip()
        if not part_stripped: continue
        is_current_part_a_heading = re.match(HEADING_SPLIT_PATTERN, part_stripped, flags=re.MULTILINE)
        if is_current_part_a_heading:
            if accumulated_text_for_section.strip():
                sub_chunks = _split_text_block_sliding_window(accumulated_text_for_section, pertemuan_id, pertemuan_judul, current_heading, MAX_CHUNK_SIZE_CHARS, CHUNK_OVERLAP_CHARS)
                for sc_idx, sc in enumerate(sub_chunks): sc['chunk_id'] = f"p{pertemuan_id}_s{i}_sc{sc_idx}"; processed_chunks.append(sc) # ID chunk yang lebih unik
            current_heading = part_stripped; accumulated_text_for_section = ""
        else: accumulated_text_for_section += part_stripped + "\n"
    if accumulated_text_for_section.strip():
        sub_chunks = _split_text_block_sliding_window(accumulated_text_for_section, pertemuan_id, pertemuan_judul, current_heading, MAX_CHUNK_SIZE_CHARS, CHUNK_OVERLAP_CHARS)
        for sc_idx, sc in enumerate(sub_chunks): sc['chunk_id'] = f"p{pertemuan_id}_s_last_sc{sc_idx}"; processed_chunks.append(sc)
    return processed_chunks

def get_text_embeddings_from_model(list_of_chunk_texts, embedding_model_instance):
    if not list_of_chunk_texts: print("Tidak ada teks untuk di-embed."); return np.array([])
    try:
        print(f"Memulai proses embedding untuk {len(list_of_chunk_texts)} chunk teks...")
        embeddings = embedding_model_instance.encode(list_of_chunk_texts, show_progress_bar=True, batch_size=128)
        print(f"Proses embedding selesai. Dihasilkan {embeddings.shape[0]} embeddings dengan dimensi {embeddings.shape[1]}.")
        return embeddings
    except Exception as e: print(f"Error saat membuat embeddings: {e}"); return np.array([])

def create_and_save_faiss_index(embeddings_np_array, index_output_path):
    if embeddings_np_array.size == 0 or embeddings_np_array.ndim != 2: print("Array embedding kosong. FAISS index tidak dibuat."); return False
    dimension = embeddings_np_array.shape[1]
    try:
        print(f"Membuat FAISS index dengan dimensi {dimension}...")
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings_np_array.astype('float32'))
        faiss.write_index(index, index_output_path)
        print(f"FAISS index dengan {index.ntotal} vektor disimpan ke: {index_output_path}")
        return True
    except Exception as e: print(f"Error saat membuat/menyimpan FAISS index: {e}"); return False

def save_chunks_to_json(chunks_with_metadata, json_output_path):
    try:
        with open(json_output_path, "w", encoding="utf-8") as f:
            json.dump(chunks_with_metadata, f, ensure_ascii=False, indent=2)
        print(f"Semua chunk ({len(chunks_with_metadata)}) berhasil disimpan ke: {json_output_path}")
        return True
    except Exception as e: print(f"Error saat menyimpan chunks ke JSON: {e}"); return False

# --- Fungsi Fine-Tuning & Evaluasi Embedding ---
def load_embedding_finetuning_dataset(filepath):
    train_examples = []
    if not os.path.exists(filepath): print(f"PERINGATAN: File dataset fine-tuning '{filepath}' tidak ditemukan."); return []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line_num, line_content in enumerate(f):
            try:
                data = json.loads(line_content)
                if "query" in data and "positive_passage" in data:
                    train_examples.append(InputExample(texts=[data["query"], data["positive_passage"]]))
                else: print(f"Peringatan: Baris data tidak valid di {filepath} (baris {line_num+1})")
            except json.JSONDecodeError: print(f"Peringatan: Gagal parsing JSON di {filepath} (baris {line_num+1})")
    print(f"Berhasil memuat {len(train_examples)} contoh dari {filepath} untuk fine-tuning embedding.")
    return train_examples

def finetune_embedding_model(base_model_name, train_examples, output_path, epochs, batch_size):
    if not train_examples:
        print("Tidak ada data training. Fine-tuning embedding dilewati.")
        return None

    print(f"Memulai fine-tuning model embedding '{base_model_name}'...")
    start_time = time.time()

    os.environ["WANDB_DISABLED"] = "true"

    model = SentenceTransformer(base_model_name)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
    train_loss = losses.MultipleNegativesRankingLoss(model=model)

    os.makedirs(output_path, exist_ok=True)

    model.fit(train_objectives=[(train_dataloader, train_loss)],
              epochs=epochs,
              warmup_steps=int(len(train_dataloader) * epochs * 0.1),
              output_path=output_path,
              show_progress_bar=True,
              save_best_model=False)

    end_time = time.time()
    print(f"Fine-tuning selesai dalam {end_time - start_time:.2f} detik. Model disimpan di: {output_path}")

    try:
        return SentenceTransformer(output_path)
    except Exception as e:
        print(f"Peringatan: Gagal memuat model dari '{output_path}' setelah fine-tuning: {e}")
        print("Mengembalikan None. Periksa apakah direktori model berisi semua file yang diperlukan.")
        return None

def load_embedding_evaluation_dataset(filepath):
    eval_data = []
    if not os.path.exists(filepath): print(f"PERINGATAN: File dataset evaluasi '{filepath}' tidak ditemukan."); return []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line_num, line_content in enumerate(f):
            try:
                data = json.loads(line_content)
                if "query_id" in data and "query_text" in data and "relevant_passage_text" in data: # Ganti nama field
                    eval_data.append(data)
                else: print(f"Peringatan: Baris data evaluasi tidak valid di {filepath} (baris {line_num+1})")
            except json.JSONDecodeError: print(f"Peringatan: Gagal parsing JSON di data evaluasi {filepath} (baris {line_num+1})")
    print(f"Berhasil memuat {len(eval_data)} contoh dari {filepath} untuk evaluasi embedding.")
    return eval_data

def evaluate_retrieval_performance(embedding_model_instance, faiss_index_instance, text_chunks_list, eval_dataset, top_k):
    if not eval_dataset or embedding_model_instance is None or faiss_index_instance is None or not text_chunks_list:
        print("Input tidak lengkap untuk evaluasi retrieval. Dilewati.")
        return {"avg_recall_at_k": 0, "avg_mrr_at_k": 0, "queries_evaluated": 0}

    total_recall = 0; total_mrr = 0; queries_evaluated = 0
    for item in eval_dataset:
        query_text = item["query_text"]
        relevant_passage_ground_truth = item["relevant_passage_text"].strip().lower()

        query_embedding = embedding_model_instance.encode([query_text])[0]
        query_np_array = np.array([query_embedding]).astype('float32')

        if faiss_index_instance.d != query_np_array.shape[1]:
            print(f"  Peringatan Evaluasi: Dimensi embedding query ({query_np_array.shape[1]}) tidak cocok dengan FAISS ({faiss_index_instance.d}).")
            continue

        distances, global_indices = faiss_index_instance.search(query_np_array, min(top_k, faiss_index_instance.ntotal))

        retrieved_passages = []
        for i in global_indices[0]:
            if i != -1 and 0 <= i < len(text_chunks_list):
                retrieved_passages.append(text_chunks_list[i]["chunk_text"].strip().lower())

        found_in_top_k = False
        rank = 0
        for idx, passage in enumerate(retrieved_passages):
            if relevant_passage_ground_truth in passage: # Menggunakan 'in' untuk pencocokan substring
                if not found_in_top_k: # Hanya hitung rank pertama kali ditemukan
                    rank = idx + 1
                    total_mrr += 1.0 / rank
                found_in_top_k = True
                break # Hentikan pencarian jika sudah ditemukan

        if found_in_top_k: total_recall += 1
        queries_evaluated += 1

    avg_recall = total_recall / queries_evaluated if queries_evaluated > 0 else 0
    avg_mrr = total_mrr / queries_evaluated if queries_evaluated > 0 else 0
    return {"avg_recall_at_k": avg_recall, "avg_mrr_at_k": avg_mrr, "queries_evaluated": queries_evaluated}

def zip_model_directory(directory_path, zip_output_path):
    if not os.path.isdir(directory_path):
        print(f"Error: Direktori model '{directory_path}' tidak ditemukan. Tidak dapat membuat zip.")
        return False
    try:
        # Hapus file zip lama jika ada
        if os.path.exists(zip_output_path):
            os.remove(zip_output_path)
            print(f"File zip lama '{zip_output_path}' dihapus.")

        shutil.make_archive(zip_output_path.replace('.zip',''), 'zip', directory_path) # make_archive menambahkan .zip otomatis
        print(f"Model berhasil di-zip ke: {zip_output_path}")
        return True
    except Exception as e:
        print(f"Error saat membuat arsip zip: {e}")
        return False

In [ ]:
def generate_queries_from_passage_with_llm(passage_text, llm_model_instance, num_queries=1, max_retries=2, delay_between_retries=5):
    """
    Menghasilkan pertanyaan dari sebuah passage menggunakan LLM (Gemini).
    """
    if not llm_model_instance:
        print("  LLM model instance tidak tersedia untuk question generation.")
        return []

    prompt = f"""Anda adalah seorang ahli dalam membuat pertanyaan berdasarkan teks yang diberikan.
Berdasarkan teks berikut, buatlah {num_queries} pertanyaan yang spesifik, jelas, dan bisa dijawab secara langsung oleh teks tersebut.
Setiap pertanyaan harus berbeda. Kembalikan hanya daftar pertanyaan, masing-masing dalam baris baru, tanpa nomor atau bullet point.

Teks:
\"\"\"
{passage_text[:1500]}
\"\"\"

Pertanyaan:
"""
    # Batasi panjang passage untuk efisiensi prompt

    for attempt in range(max_retries):
        try:
            response = llm_model_instance.generate_content(prompt)
            # Cek apakah ada 'parts' dan ambil teksnya, atau langsung dari 'text'
            if hasattr(response, 'parts') and response.parts:
                generated_text = "".join(part.text for part in response.parts)
            elif hasattr(response, 'text') and response.text:
                generated_text = response.text
            else: # Jika tidak ada parts atau text, coba cek prompt_feedback
                if hasattr(response, 'prompt_feedback') and response.prompt_feedback.block_reason:
                    print(f"  Peringatan LLM QGen: Respons diblokir. Alasan: {response.prompt_feedback.block_reason_message}")
                else:
                    print(f"  Peringatan LLM QGen: Respons tidak memiliki format teks yang diharapkan. Respons: {response}")
                generated_text = ""

            questions = [q.strip() for q in generated_text.splitlines() if q.strip()]

            if questions: # Jika berhasil mendapatkan pertanyaan
                # Pastikan tidak ada pertanyaan kosong setelah strip
                questions = [q for q in questions if q]
                if questions:
                     print(f"  LLM berhasil generate {len(questions)} pertanyaan untuk passage.")
                     return questions[:num_queries] # Ambil sejumlah num_queries

            print(f"  LLM tidak menghasilkan pertanyaan yang valid (attempt {attempt + 1}/{max_retries}). Respons mentah: '{generated_text}'")

        except Exception as e:
            print(f"  Error saat memanggil LLM untuk question generation (attempt {attempt + 1}/{max_retries}): {e}")

        if attempt < max_retries - 1:
            print(f"  Menunggu {delay_between_retries} detik sebelum mencoba lagi...")
            time.sleep(delay_between_retries)

    return []


def create_embedding_finetune_dataset_with_llm(
    all_processed_chunks,
    llm_model_instance_for_qgen,
    num_queries_per_chunk=1,
    output_filepath=None,
    max_chunks_to_process=None # Opsional: batasi jumlah chunk untuk diolah (untuk testing)
):
    """
    Membuat dataset fine-tuning (query, positive_passage) menggunakan LLM untuk membuat query.
    Menyimpan hasilnya ke output_filepath jika diberikan.
    """
    train_examples_data = []

    chunks_to_process = all_processed_chunks
    if max_chunks_to_process is not None:
        chunks_to_process = all_processed_chunks[:max_chunks_to_process]
        print(f"Memproses {max_chunks_to_process} chunk pertama untuk pembuatan dataset fine-tuning dengan LLM.")

    processed_count = 0
    for i, chunk_data in enumerate(chunks_to_process):
        print(f"Memproses chunk {i+1}/{len(chunks_to_process)} untuk Q-Gen...")
        passage = chunk_data["chunk_text"]
        generated_queries = generate_queries_from_passage_with_llm(
            passage, llm_model_instance_for_qgen, num_queries_per_chunk
        )
        for query in generated_queries:
            train_examples_data.append({"query": query, "positive_passage": passage})

        processed_count +=1
        # Opsional: tambahkan delay kecil untuk menghindari rate limit API
        if processed_count % 10 == 0: # Setiap 10 chunk
             time.sleep(1)


    if output_filepath and train_examples_data:
        try:
            with open(output_filepath, 'w', encoding='utf-8') as f:
                for example in train_examples_data:
                    f.write(json.dumps(example) + '\n')
            print(f"Dataset fine-tuning embedding ({len(train_examples_data)} contoh) berhasil dibuat dan disimpan di: {output_filepath}")
        except Exception as e:
            print(f"Error saat menyimpan dataset fine-tuning ke {output_filepath}: {e}")
    elif not train_examples_data:
        print("Tidak ada contoh fine-tuning yang berhasil dibuat dengan LLM.")

    # Mengembalikan format InputExample jika ingin langsung digunakan untuk training
    # atau biarkan skrip utama yang memanggil `load_embedding_finetuning_dataset`
    # Untuk konsistensi, kita simpan ke file dan biarkan `load_embedding_finetuning_dataset` yang memuatnya.
    return train_examples_data # Mengembalikan list of dicts

In [ ]:
from google.colab import userdata

print("Memulai Persiapan Data, Fine-Tuning, dan Evaluasi Embedding...")

# 1. Parse outline & Buat Chunks Materi (dilakukan sekali)
print(f"\n--- Langkah 1: Memproses Materi Kuliah ---")
daftar_pertemuan = parse_outline(OUTLINE_FILE)
all_processed_chunks_with_metadata = []

llm_for_qgen = None
GEMINI_API_KEY_FOR_PREP = userdata.get('GEMINI')

if GEMINI_API_KEY_FOR_PREP:
    try:
        genai.configure(api_key=GEMINI_API_KEY_FOR_PREP)
        print("Gemini API berhasil dikonfigurasi (atau dikonfigurasi ulang) dengan API Key dari userdata.")

        llm_for_qgen = genai.GenerativeModel('gemini-1.5-flash-latest')
        print("Model LLM ('gemini-1.5-flash-latest') untuk Question Generation berhasil dimuat.")

    except Exception as e:
        print(f"Peringatan: Terjadi kesalahan saat mengkonfigurasi atau memuat model Gemini untuk QGen: {e}")
        llm_for_qgen = None
else:
    print("PERINGATAN: Secret 'GEMINI' (API Key) tidak ditemukan di Colab userdata. Question Generation dengan LLM akan dilewati.")


Memulai Persiapan Data, Fine-Tuning, dan Evaluasi Embedding...

--- Langkah 1: Memproses Materi Kuliah ---
Berhasil mem-parsing 4 pertemuan dengan file materi dari outline.
Gemini API berhasil dikonfigurasi (atau dikonfigurasi ulang) dengan API Key dari userdata.
Model LLM ('gemini-1.5-flash-latest') untuk Question Generation berhasil dimuat.


In [ ]:
if daftar_pertemuan:
    for pertemuan_info in daftar_pertemuan:
        file_materi_abs_path = os.path.join(BASE_DIR, pertemuan_info.get('file_materi',''))
        materi_text = read_material_text(file_materi_abs_path)
        if materi_text:
            chunks_for_this_pertemuan = chunk_material_heading_aware(materi_text, pertemuan_info.get('id'), pertemuan_info.get('judul'))
            all_processed_chunks_with_metadata.extend(chunks_for_this_pertemuan)
    print(f"Total chunk materi yang diproses: {len(all_processed_chunks_with_metadata)}")
else:
    print("Gagal mem-parsing outline atau tidak ada materi. Proses dihentikan.")
    exit()

if not all_processed_chunks_with_metadata:
    print("Tidak ada chunk materi yang berhasil dibuat. Proses dihentikan.")
    exit()

list_of_chunk_texts_for_embedding = [chunk['chunk_text'] for chunk in all_processed_chunks_with_metadata]

Total chunk materi yang diproses: 56


In [ ]:
if not os.path.exists(EMBEDDING_FINETUNING_DATASET_FILE) and llm_for_qgen and all_processed_chunks_with_metadata:
    print(f"\n--- Langkah Opsional: Membuat Dataset Fine-tuning Embedding dengan LLM ---")
    print(f"File {EMBEDDING_FINETUNING_DATASET_FILE} tidak ditemukan. Mencoba membuat secara otomatis...")
    create_embedding_finetune_dataset_with_llm(
        all_processed_chunks_with_metadata,
        llm_for_qgen,
        num_queries_per_chunk=1, # Buat 1 pertanyaan per chunk
        output_filepath=EMBEDDING_FINETUNING_DATASET_FILE,
        max_chunks_to_process=5 # Hapus atau naikkan angka ini untuk memproses semua chunk
                                  # Gunakan angka kecil untuk testing agar tidak lama dan boros API call.
    )
elif not llm_for_qgen and not os.path.exists(EMBEDDING_FINETUNING_DATASET_FILE):
    print(f"PERINGATAN: LLM untuk Q-Gen tidak tersedia DAN file {EMBEDDING_FINETUNING_DATASET_FILE} tidak ada. Fine-tuning akan dilewati.")

In [ ]:
# 2. Muat dataset untuk fine-tuning dan evaluasi
print(f"\n--- Langkah 2: Memuat Dataset Tambahan ---")
embedding_finetune_examples = load_embedding_finetuning_dataset(EMBEDDING_FINETUNING_DATASET_FILE)
embedding_evaluation_examples = load_embedding_evaluation_dataset(EMBEDDING_EVALUATION_SET_FILE)
metrics_base = None
metrics_finetuned = None

# --- BASELINE MODEL: Pembuatan Artefak & Evaluasi ---
print(f"\n--- Langkah 3: Memproses dengan Model Embedding Dasar ({BASE_EMBEDDING_MODEL_NAME}) ---")
base_embedding_model_instance = SentenceTransformer(BASE_EMBEDDING_MODEL_NAME)
base_document_embeddings = get_text_embeddings_from_model(list_of_chunk_texts_for_embedding, base_embedding_model_instance)

base_faiss_created = False
if base_document_embeddings.size > 0:
    base_faiss_created = create_and_save_faiss_index(base_document_embeddings, OUTPUT_FAISS_INDEX_BASE)
    save_chunks_to_json(all_processed_chunks_with_metadata, OUTPUT_JSON_CHUNKS_BASE) # Simpan chunks yang sesuai

if base_faiss_created and embedding_evaluation_examples:
    print(f"\n--- Langkah 3.1: Evaluasi Performa Retrieval Model Dasar ---")
    # Muat FAISS index yang baru dibuat untuk evaluasi
    base_faiss_index_for_eval = faiss.read_index(OUTPUT_FAISS_INDEX_BASE)
    metrics_base = evaluate_retrieval_performance(
        base_embedding_model_instance, base_faiss_index_for_eval, all_processed_chunks_with_metadata,
        embedding_evaluation_examples, EVALUATION_TOP_K
    )
    print(f"Metrik Model Dasar: Recall@{EVALUATION_TOP_K}={metrics_base['avg_recall_at_k']:.4f}, MRR@{EVALUATION_TOP_K}={metrics_base['avg_mrr_at_k']:.4f} ({metrics_base['queries_evaluated']} queries)")

# --- FINE-TUNING EMBEDDING MODEL ---
print(f"\n--- Langkah 4: Fine-Tuning Model Embedding ---")
finetuned_embedding_model_instance = None
if embedding_finetune_examples:
    finetuned_embedding_model_instance = finetune_embedding_model(
        BASE_EMBEDDING_MODEL_NAME, embedding_finetune_examples,
        FINETUNED_EMBEDDING_MODEL_SAVE_PATH, FINETUNE_EPOCHS, FINETUNE_BATCH_SIZE
    )
else:
    print("Tidak ada data fine-tuning, menggunakan model dasar sebagai model akhir.")

# Tentukan model mana yang akan digunakan untuk artefak final (yang akan dipakai aplikasi)
active_embedding_model_for_final_artifacts = finetuned_embedding_model_instance if finetuned_embedding_model_instance else base_embedding_model_instance
active_model_name_for_log = FINETUNED_EMBEDDING_MODEL_SAVE_PATH if finetuned_embedding_model_instance else BASE_EMBEDDING_MODEL_NAME

print(f"\n--- Langkah 5: Membuat Artefak RAG Final dengan Model '{active_model_name_for_log}' ---")
final_document_embeddings = get_text_embeddings_from_model(list_of_chunk_texts_for_embedding, active_embedding_model_for_final_artifacts)

final_faiss_created = False
if final_document_embeddings.size > 0:
    final_faiss_created = create_and_save_faiss_index(final_document_embeddings, OUTPUT_FAISS_INDEX_FINETUNED)
    save_chunks_to_json(all_processed_chunks_with_metadata, OUTPUT_JSON_CHUNKS_FINETUNED)

if final_faiss_created and finetuned_embedding_model_instance and embedding_evaluation_examples:
    print(f"\n--- Langkah 5.1: Evaluasi Performa Retrieval Model Fine-Tuned ---")
    # Muat FAISS index fine-tuned yang baru dibuat untuk evaluasi
    finetuned_faiss_index_for_eval = faiss.read_index(OUTPUT_FAISS_INDEX_FINETUNED)
    metrics_finetuned = evaluate_retrieval_performance(
        finetuned_embedding_model_instance, finetuned_faiss_index_for_eval, all_processed_chunks_with_metadata,
        embedding_evaluation_examples, EVALUATION_TOP_K
    )
    print(f"Metrik Model Fine-Tuned: Recall@{EVALUATION_TOP_K}={metrics_finetuned['avg_recall_at_k']:.4f}, MRR@{EVALUATION_TOP_K}={metrics_finetuned['avg_mrr_at_k']:.4f} ({metrics_finetuned['queries_evaluated']} queries)")


--- Langkah 2: Memuat Dataset Tambahan ---
Berhasil memuat 5 contoh dari dataset/SistemOperasi/embedding_finetuning_examples.jsonl untuk fine-tuning embedding.
PERINGATAN: File dataset evaluasi 'dataset/SistemOperasi/embedding_evaluation_set.jsonl' tidak ditemukan.

--- Langkah 3: Memproses dengan Model Embedding Dasar (all-MiniLM-L6-v2) ---
Memulai proses embedding untuk 56 chunk teks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Proses embedding selesai. Dihasilkan 56 embeddings dengan dimensi 384.
Membuat FAISS index dengan dimensi 384...
FAISS index dengan 56 vektor disimpan ke: dataset/SistemOperasi/vector_store_base.index
Semua chunk (56) berhasil disimpan ke: dataset/SistemOperasi/processed_chunks_metadata_base.json

--- Langkah 4: Fine-Tuning Model Embedding ---
Memulai fine-tuning model embedding 'all-MiniLM-L6-v2'...


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


Fine-tuning selesai dalam 3.08 detik. Model disimpan di: dataset/SistemOperasi/finetuned_embedding_model_sistem_operasi

--- Langkah 5: Membuat Artefak RAG Final dengan Model 'dataset/SistemOperasi/finetuned_embedding_model_sistem_operasi' ---
Memulai proses embedding untuk 56 chunk teks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Proses embedding selesai. Dihasilkan 56 embeddings dengan dimensi 384.
Membuat FAISS index dengan dimensi 384...
FAISS index dengan 56 vektor disimpan ke: dataset/SistemOperasi/vector_store_finetuned.index
Semua chunk (56) berhasil disimpan ke: dataset/SistemOperasi/processed_chunks_metadata_finetuned.json


In [ ]:
# --- KOMPARASI HASIL EVALUASI ---
print(f"\n--- Langkah 6: Komparasi Performa Retrieval (Jika Ada) ---")
if metrics_base and metrics_finetuned:
    print(f"Model Dasar ({BASE_EMBEDDING_MODEL_NAME}):")
    print(f"  Recall@{EVALUATION_TOP_K}: {metrics_base['avg_recall_at_k']:.4f}")
    print(f"  MRR@{EVALUATION_TOP_K}:    {metrics_base['avg_mrr_at_k']:.4f}")
    print(f"  ({metrics_base['queries_evaluated']} kueri dievaluasi)")

    print(f"\nModel Fine-Tuned ({FINETUNED_EMBEDDING_MODEL_SAVE_PATH}):")
    print(f"  Recall@{EVALUATION_TOP_K}: {metrics_finetuned['avg_recall_at_k']:.4f}")
    print(f"  MRR@{EVALUATION_TOP_K}:    {metrics_finetuned['avg_mrr_at_k']:.4f}")
    print(f"  ({metrics_finetuned['queries_evaluated']} kueri dievaluasi)")
    if metrics_finetuned['avg_recall_at_k'] > metrics_base['avg_recall_at_k']:
        print("\nKESIMPULAN: Model Fine-Tuned menunjukkan peningkatan Recall!")
    if metrics_finetuned['avg_mrr_at_k'] > metrics_base['avg_mrr_at_k']:
        print("KESIMPULAN: Model Fine-Tuned menunjukkan peningkatan MRR!")
elif metrics_base:
    print(f"Hanya metrik model dasar yang tersedia: Recall@{EVALUATION_TOP_K}={metrics_base['avg_recall_at_k']:.4f}, MRR@{EVALUATION_TOP_K}={metrics_base['avg_mrr_at_k']:.4f}")
else:
    print("Tidak ada metrik evaluasi yang tersedia.")


--- Langkah 6: Komparasi Performa Retrieval (Jika Ada) ---
Tidak ada metrik evaluasi yang tersedia.


In [ ]:
if finetuned_embedding_model_instance is not None:
  print(f"\n--- Langkah 7: Menyimpan Model Fine-Tuned ---")
  try:
    # Model sudah disimpan ke FINETUNED_EMBEDDING_MODEL_SAVE_PATH oleh fungsi finetune_embedding_model
    print(f"Model fine-tuned sudah tersimpan di direktori: {FINETUNED_EMBEDDING_MODEL_SAVE_PATH}")
    # Anda bisa menambahkan verifikasi di sini jika perlu
    if os.path.exists(FINETUNED_EMBEDDING_MODEL_SAVE_PATH):
        print("Direktori model fine-tuned berhasil diverifikasi.")
    else:
        print("ERROR: Direktori model fine-tuned tidak ditemukan setelah proses fine-tuning.")

  except Exception as e:
    print(f"Error saat mencoba memastikan lokasi model fine-tuned: {e}")
else:
  print("\nTidak ada model fine-tuned untuk disimpan tanpa zip (karena fine-tuning dilewati).")

print("\nProses selesai.")



--- Langkah 7: Menyimpan Model Fine-Tuned ---
Model fine-tuned sudah tersimpan di direktori: dataset/SistemOperasi/finetuned_embedding_model_sistem_operasi
Direktori model fine-tuned berhasil diverifikasi.

Proses selesai.


In [ ]:
# prompt: zip dataset folder

# 8. Zip folder dataset
print(f"\n--- Langkah 8: Meng-zip Folder Dataset ---")
DATASET_FOLDER_TO_ZIP = "dataset" # Folder yang akan di-zip
DATASET_ZIP_OUTPUT_PATH = "dataset_zipped_for_download.zip" # Nama file zip output

if os.path.isdir(DATASET_FOLDER_TO_ZIP):
    print(f"Meng-zip folder '{DATASET_FOLDER_TO_ZIP}'...")
    try:
        # Hapus file zip lama jika ada
        if os.path.exists(DATASET_ZIP_OUTPUT_PATH):
            os.remove(DATASET_ZIP_OUTPUT_PATH)
            print(f"File zip dataset lama '{DATASET_ZIP_OUTPUT_PATH}' dihapus.")

        shutil.make_archive(DATASET_ZIP_OUTPUT_PATH.replace('.zip', ''), 'zip', DATASET_FOLDER_TO_ZIP)
        print(f"Folder dataset berhasil di-zip ke: {DATASET_ZIP_OUTPUT_PATH}")

        # Verifikasi bahwa file zip dibuat
        if os.path.exists(DATASET_ZIP_OUTPUT_PATH):
            print(f"Ukuran file zip: {os.path.getsize(DATASET_ZIP_OUTPUT_PATH)} bytes")
        else:
            print(f"ERROR: File zip '{DATASET_ZIP_OUTPUT_PATH}' tidak ditemukan setelah proses zip.")

    except Exception as e:
        print(f"Error saat membuat arsip zip folder dataset: {e}")
else:
    print(f"Error: Direktori dataset '{DATASET_FOLDER_TO_ZIP}' tidak ditemukan. Tidak dapat membuat zip.")

print("\nProses selesai.")



--- Langkah 8: Meng-zip Folder Dataset ---
Meng-zip folder 'dataset'...
Folder dataset berhasil di-zip ke: dataset_zipped_for_download.zip
Ukuran file zip: 83818331 bytes

Proses selesai.
